In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from dateutil.parser import parse
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, TensorDataset
import matplotlib.pyplot as plt

In [2]:
###TODO: expand to full data, figure out how to visualize the difference in compounds or just train multiple neural networks on each compound, fix the size error idk why thats happening

In [3]:
filenames = ["LapWeather_Australian Grand Prix.csv","LapWeather_Austrian Grand Prix.csv","LapWeather_Azerbaijan Grand Prix.csv","LapWeather_Bahrain Grand Prix.csv","LapWeather_Belgian Grand Prix.csv","LapWeather_Brazilian Grand Prix.csv","LapWeather_British Grand Prix.csv","LapWeather_Canadian Grand Prix.csv", "LapWeather_Chinese Grand Prix.csv","LapWeather_French Grand Prix.csv","LapWeather_German Grand Prix.csv","LapWeather_Hungarian Grand Prix.csv","LapWeather_Italian Grand Prix.csv","LapWeather_Japanese Grand Prix.csv","LapWeather_Mexican Grand Prix.csv","LapWeather_Monaco Grand Prix.csv","LapWeather_Russian Grand Prix.csv","LapWeather_Singapore Grand Prix.csv","LapWeather_Spanish Grand Prix.csv","LapWeather_United States Grand Prix.csv"]
combined_csv = pd.concat([pd.read_csv(f) for f in filenames ])
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [4]:
df = pd.read_csv('combined_csv.csv')
df = df.drop(['Time','DriverNumber','LapStartDate', 'Unnamed: 0', 'PitOutTime', 'PitInTime', 'Sector1Time','Sector2Time','Sector3Time','Sector1SessionTime','Sector2SessionTime','Sector3SessionTime','SpeedI1','SpeedI2','SpeedST','IsPersonalBest','FreshTyre','SpeedFL','LapStartTime','Team','Driver','TrackStatus','IsAccurate'], axis=1)
df = df.dropna()

dfOutput = df['LapTime']
train_targets = dfOutput.to_numpy()

for i in range(len(train_targets)):
    train_targets[i] = train_targets[i].replace('0 days ', '')
# print(train_targets)
actual_train_targets = []
for time in train_targets:
    td = parse(time) - parse('00:00:00')
    seconds = td.total_seconds()
    actual_train_targets.append(seconds)
# print(actual_train_targets)
dfLapTime = pd.DataFrame(actual_train_targets)

df['LapTime'] = dfLapTime
df = df.dropna()
cols = ['LapTime']
Q1 = df[cols].quantile(0.25)
Q3 = df[cols].quantile(0.75)
IQR = Q3-Q1
df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
dfInput = df.drop(['LapTime'], axis=1)
dfInput = dfInput.replace({'SUPERHARD':1, 'HARD':2, 'MEDIUM':3, 'SOFT':4,'SUPERSOFT': 5, 'ULTRASOFT': 6,'HYPERSOFT':7,"INTERMEDIATE":0,"WET":-1})
train_inputs = dfInput.to_numpy()
inputs_array = train_inputs.astype('float64')
dfOutput = df['LapTime']

targets_array = dfOutput.to_numpy()
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

print(inputs_array)
print(targets_array)


[[  2.    6.    4.  ...  38.2 296.    3.8]
 [  3.    6.    5.  ...  36.7 289.    4.3]
 [  4.    6.    6.  ...  36.8 255.    2.9]
 ...
 [  6.    5.    9.  ...  30.3  87.    3. ]
 [  7.    5.   10.  ...  30.5  89.    2.7]
 [  8.    5.   11.  ...  30.3  99.    2.2]]
[ 90.177  89.61   89.54  ... 100.552 100.83  103.413]


In [5]:
dataset = TensorDataset(inputs, targets)

In [6]:
val_percent = 0.2
num_rows = len(dfInput.index)
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

In [7]:
batch_size = 9
train_loader = DataLoader(train_ds, batch_size, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_ds, batch_size, num_workers = 0)

In [8]:
input_size = len(inputs[0])
output_size = 1

In [9]:
class MultiLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(11, 36),
            nn.ReLU(),
            nn.Linear(36,18),
            nn.ReLU(),
            nn.Linear(18,1)
        )
        
    def forward(self, xb):
        out = self.layers(xb)                       
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(input = out, target = targets, size_average = None, reduce = None, reduction = 'mean')  
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(input = out, target = targets, size_average = None, reduce = None, reduction = 'mean')    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [10]:
model = MultiLinearRegressionModel()

In [11]:
list(model.parameters())

[Parameter containing:
 tensor([[-2.4807e-01,  2.9681e-01, -2.4541e-01, -2.5394e-01, -2.3929e-01,
           2.5242e-01,  7.0965e-02, -2.6725e-01, -2.3794e-01, -1.7920e-01,
          -1.1500e-01],
         [ 4.0638e-02, -2.6811e-01,  2.8840e-01, -8.4375e-02,  2.9007e-01,
           2.6344e-01,  1.3608e-01,  2.5505e-03,  2.5329e-01,  1.9598e-01,
           5.3128e-02],
         [-1.4135e-01, -3.2963e-02, -1.5339e-01,  1.7873e-01,  2.3644e-01,
          -1.3322e-01, -2.1399e-01,  6.4566e-02,  1.7005e-01, -2.4731e-01,
          -2.7327e-01],
         [-6.6129e-02,  1.2752e-01, -1.2415e-01,  2.8823e-01,  2.9029e-01,
          -1.2603e-01,  2.5803e-01, -2.1049e-01, -2.8530e-01, -1.1630e-01,
           7.7897e-02],
         [ 1.4937e-01, -5.7018e-02,  1.9241e-01, -8.1632e-03,  2.7937e-01,
           3.9424e-02, -2.7624e-01, -1.3343e-01, -1.3487e-01, -1.6709e-01,
           2.9074e-01],
         [-2.4827e-01, -1.8956e-01,  1.4760e-01, -2.0358e-01, -3.1288e-02,
           1.6744e-01,  2.4943e-

In [12]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [13]:
result = evaluate(model, val_loader)
print(result)

C:\Temp/ipykernel_6468/1372700796.py:29: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(input = out, target = targets, size_average = None, reduce = None, reduction = 'mean')
C:\Temp/ipykernel_6468/1372700796.py:29: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(input = out, target = targets, size_average = None, reduce = None, reduction = 'mean')


{'val_loss': 85.80325317382812}


In [ ]:
epochs = 1000
lr = 1e-7
history1 = fit(epochs, lr, model, train_loader, val_loader)

C:\Temp/ipykernel_6468/1372700796.py:21: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(input = out, target = targets, size_average = None, reduce = None, reduction = 'mean')
C:\Temp/ipykernel_6468/1372700796.py:21: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(input = out, target = targets, size_average = None, reduce = None, reduction = 'mean')
C:\Temp/ipykernel_6468/1372700796.py:29: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(input = ou

Epoch [20], val_loss: 11.6690
Epoch [40], val_loss: 11.6450
Epoch [60], val_loss: 11.6357
Epoch [80], val_loss: 11.6316
Epoch [100], val_loss: 11.6297
Epoch [120], val_loss: 11.6243
Epoch [140], val_loss: 11.6223
Epoch [160], val_loss: 11.6181


In [ ]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                
    prediction = predictions[0].detach()
    return target, prediction

In [ ]:
input, target = val_ds[11]
predict_single(input, target, model)

In [ ]:
input, target = val_ds[50]
predict_single(input, target, model)

In [ ]:
targets = []
predictions = []
for i in range(len(val_ds)):
    input, target = val_ds[i]
    target, prediction = predict_single(input, target, model)
    targets.append(target)
    predictions.append(prediction)


In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

ax.scatter(range(len(predictions)),targets)
ax.scatter(range(len(predictions)), predictions)
